# Minimise the number of synaptic operations
As described in [Sorbaro et al. 2020](https://www.frontiersin.org/articles/10.3389/fnins.2020.00662/full), it can be beneficial for power consumption and latency in the network to reduce the number of synaptic operations (synops). This number is essentially the output activation multiplied by the number of outward connections (fan-out) to the next layer. We describe how using Sinabs' synops counters it's possible to easily add a term to your loss function which then can be minimised.

## Training an ANN with fewer synops
Let's start by defining our ANN. Keep in mind that we use NeuromorphicRelus here as we need to discretize the output in the forward pass to simulate the number of spikes that layer would emit. In the backward pass the derivative of the ReLU function is used. 

In [ ]:
import torch
import torch.nn as nn
import sinabs
import sinabs.layers as sl


ann = nn.Sequential(
    nn.Conv2d(1, 16, 5, bias=False),
    sl.NeuromorphicReLU(),
    sl.SumPool2d(2),
    nn.Conv2d(16, 32, 5, bias=False),
    sl.NeuromorphicReLU(),
    sl.SumPool2d(2),
    nn.Conv2d(32, 120, 4, bias=False),
    sl.NeuromorphicReLU(),
    nn.Flatten(),
    nn.Linear(120, 10, bias=False),
)

Let's apply a SynopsCounter to our ANN to track how many synaptic operations it would need in an SNN.

In [ ]:
batch_size = 5

synops_counter_ann = sinabs.SynOpCounter(ann.modules(), sum_activations=True)

rand_input = torch.rand((batch_size, 1, 28, 28))
ann(rand_input)
print(f"Synops after feeding input: {synops_counter_ann()}")

## Training an SNN with fewer synops
Let's start by defining your SNN model.

In [ ]:
class SNN(nn.Sequential):
    def __init__(self, batch_size):
        super().__init__(
            sl.FlattenTime(),
            nn.Conv2d(1, 16, 5, bias=False),
            sl.IAFSqueeze(batch_size=batch_size),
            sl.SumPool2d(2),
            nn.Conv2d(16, 32, 5, bias=False),
            sl.IAFSqueeze(batch_size=batch_size),
            sl.SumPool2d(2),
            nn.Conv2d(32, 120, 4, bias=False),
            sl.IAFSqueeze(batch_size=batch_size),
            nn.Flatten(),
            nn.Linear(120, 10, bias=False),
            sl.IAFSqueeze(batch_size=batch_size),
            sl.UnflattenTime(batch_size=batch_size),
        )

    @property
    def spiking_layers(self):
        return [layer for layer in self.net.children() if isinstance(layer, sl.StatefulLayer)]

    def reset_states(self):
        for layer in self.spiking_layers:
            layer.reset_states()


batch_size = 5
snn = SNN(batch_size=batch_size)
snn

If we apply a SynopsLossCounter to the model, we'll be able to track the number of synops as we feed new inputs to the model.

In [ ]:
synops_counter = sinabs.SNNSynOpCounter(snn)
print(f"Synops before feeding input: {synops_counter.get_total_synops()}")

rand_input_spikes = (torch.rand((batch_size, 10, 1, 28, 28)) < 0.1).float()
y_hat = snn(rand_input_spikes)
print(f"Synops after feeding input: {synops_counter.get_total_synops()}")

You can also get a more detailed count for each layer.

In [ ]:
synops_counter.get_synops()

Once we have can calculate the total synops, we might want to choose a target synops number in order to decrease power consumption. As a rule of thumb we're going to take half of the number of initial synops as constant target.

In [ ]:
# done once
target_synops = synops_counter.get_total_synops() / 2

# in your training loop
synops = synops_counter.get_total_synops()
synops_loss = (target_synops - synops).square() / target_synops.square()

loss = y_hat.sum(1) + synops_loss